In [2]:
import pandas as pd
import numpy as np

model_input_path = "../data/model_input.parquet"

df_model_input = pd.read_parquet(model_input_path)

In [3]:
# --- Característica 1: Día de la Semana ---
# El ciclo tiene 7 periodos (de 0 a 6)
df_model_input['day_of_week_sin'] = np.sin(2 * np.pi * df_model_input['day_of_week'] / 7)
df_model_input['day_of_week_cos'] = np.cos(2 * np.pi * df_model_input['day_of_week'] / 7)

# --- Característica 2: Hora del Día ---
# El ciclo tiene 24 periodos (de 0 a 23)
df_model_input['hour_sin'] = np.sin(2 * np.pi * df_model_input['hour'] / 24)
df_model_input['hour_cos'] = np.cos(2 * np.pi * df_model_input['hour'] / 24)

# --- Característica 3: Mes del Año ---
# El ciclo tiene 12 periodos (de 1 a 12)
df_model_input['month_sin'] = np.sin(2 * np.pi * df_model_input['month'] / 12)
df_model_input['month_cos'] = np.cos(2 * np.pi * df_model_input['month'] / 12)


# Veamos las nuevas columnas que hemos creado
print(df_model_input[['day_of_week', 'day_of_week_sin', 'day_of_week_cos']].head())

   day_of_week  day_of_week_sin  day_of_week_cos
0            6        -0.781831          0.62349
1            6        -0.781831          0.62349
2            6        -0.781831          0.62349
3            6        -0.781831          0.62349
4            6        -0.781831          0.62349


In [5]:
# --- Característica 4: Demanda en la Hora Anterior (Lag de 1 hora) ---

# Ordenamos los datos para asegurarnos de que el retraso se calcula correctamente
# para cada ruta de autobús por separado.
df_model_input = df_model_input.sort_values(by=['bus_route', 'hour_timestamp'])

# Creamos la nueva columna. .shift(1) desplaza los datos una fila hacia abajo.
# Lo agrupamos por 'bus_route' para que el lag no "salte" de una ruta a otra.
df_model_input['ridership_lag_1h'] = df_model_input.groupby('bus_route', observed=False)['ridership_total'].shift(1)

# Veamos el resultado. Fíjate en la primera fila de cada ruta.
print(df_model_input[['bus_route', 'hour_timestamp', 'ridership_total', 'ridership_lag_1h']].head())

  bus_route      hour_timestamp  ridership_total  ridership_lag_1h
0       M15 2023-01-01 00:00:00              323               NaN
1       M15 2023-01-01 01:00:00              300             323.0
2       M15 2023-01-01 02:00:00              144             300.0
3       M15 2023-01-01 03:00:00              101             144.0
4       M15 2023-01-01 04:00:00               38             101.0


In [6]:
# --- Manejo de Valores Faltantes ---

# Guardamos el número de filas antes de limpiar para ver el cambio
filas_antes = len(df_model_input)
print(f"Número de filas antes de limpiar los NaN: {filas_antes}")

# Eliminamos todas las filas que contengan cualquier valor NaN
df_model_input = df_model_input.dropna()

filas_despues = len(df_model_input)
print(f"Número de filas después de limpiar los NaN: {filas_despues}")
print(f"Se eliminaron {filas_antes - filas_despues} filas.")

Número de filas antes de limpiar los NaN: 19632
Número de filas después de limpiar los NaN: 19629
Se eliminaron 3 filas.


In [7]:
# --- Característica 5: Demanda del Día Anterior (Lag de 24 horas) ---

# Usamos .shift(24) para obtener el valor de ridership de exactamente 24 horas antes
# para la misma ruta de autobús.
df_model_input['ridership_lag_24h'] = df_model_input.groupby('bus_route', observed=False)['ridership_total'].shift(24)

# --- Manejo de Valores Faltantes ---

# Ahora, vamos a limpiar los NaNs generados por este nuevo lag
filas_antes = len(df_model_input)
print(f"Filas antes de limpiar el lag de 24h: {filas_antes}")

df_model_input = df_model_input.dropna()

filas_despues = len(df_model_input)
print(f"Filas después de limpiar el lag de 24h: {filas_despues}")
print(f"Se eliminaron {filas_antes - filas_despues} filas.")

# Veamos el resultado
print("\nDataFrame con lag de 1h y 24h:")
print(df_model_input[['bus_route', 'hour_timestamp', 'ridership_total', 'ridership_lag_1h', 'ridership_lag_24h']].head())

Filas antes de limpiar el lag de 24h: 19629
Filas después de limpiar el lag de 24h: 19557
Se eliminaron 72 filas.

DataFrame con lag de 1h y 24h:
   bus_route      hour_timestamp  ridership_total  ridership_lag_1h  \
25       M15 2023-01-02 01:00:00               45              92.0   
26       M15 2023-01-02 02:00:00                8              45.0   
27       M15 2023-01-02 03:00:00               12               8.0   
28       M15 2023-01-02 04:00:00               51              12.0   
29       M15 2023-01-02 05:00:00              142              51.0   

    ridership_lag_24h  
25              300.0  
26              144.0  
27              101.0  
28               38.0  
29               60.0  


In [8]:
# --- Característica 6: Demanda de la semana anterior ---


df_model_input['ridership_lag_1_week'] = df_model_input.groupby('bus_route', observed=False)['ridership_total'].shift(24*7)

# --- Manejo de Valores Faltantes ---

# Ahora, vamos a limpiar los NaNs generados por este nuevo lag
filas_antes = len(df_model_input)
print(f"Filas antes de limpiar: {filas_antes}")

df_model_input = df_model_input.dropna()

filas_despues = len(df_model_input)
print(f"Filas después de limpiar el lag: {filas_despues}")
print(f"Se eliminaron {filas_antes - filas_despues} filas.")

# Veamos el resultado
print(df_model_input[['bus_route', 'hour_timestamp', 'ridership_total', 'ridership_lag_24h', 'ridership_lag_1_week']].head())

Filas antes de limpiar: 19557
Filas después de limpiar el lag: 19053
Se eliminaron 504 filas.
    bus_route      hour_timestamp  ridership_total  ridership_lag_24h  \
193       M15 2023-01-09 01:00:00               26               71.0   
194       M15 2023-01-09 02:00:00               11               46.0   
195       M15 2023-01-09 03:00:00               23               33.0   
196       M15 2023-01-09 04:00:00               76               34.0   
197       M15 2023-01-09 05:00:00              154               86.0   

     ridership_lag_1_week  
193                  45.0  
194                   8.0  
195                  12.0  
196                  51.0  
197                 142.0  
